In [1]:
import itertools 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import pickle

In [2]:
# utils 

def generateArrangementsOfsizeN(nucleotideAlphabetList, subSeqSize):
    arrangement = []

    for word in itertools.product(nucleotideAlphabetList, repeat=subSeqSize):
        newString = ''.join(word)
        arrangement.append(newString)

    sorted_arrangement = sorted(arrangement)

    return sorted_arrangement


def intToBinaryCharArray(number, numOfBits):
    return list(bin(number)[2:].zfill(numOfBits))

def binaryCharArrayToBinaryIntArray(binaryCharArray):
    binaryIntArray = []
    for c in binaryCharArray:
        binaryIntArray.append(int(c))
    return binaryIntArray


def numToBinaryArray(num, numOfBits):
    l0 = intToBinaryCharArray(num, numOfBits)
    l1 = binaryCharArrayToBinaryIntArray(l0)
    return l1

def createDictForNucleotidesAlphabet(nucleotideAlphabet, subSeqSize, numOfBits):
    gen_dict={}
    subSeqs = generateArrangementsOfsizeN(nucleotideAlphabet, subSeqSize)
    for i in range(0, len(subSeqs)):
        gen_dict[subSeqs[i]] = numToBinaryArray(i+1, numOfBits)
    
    return gen_dict

def generateSubSeqsFromSeq(seq, windowSize = 3, strideSize = 1, printInfo = False):
    #windowSize = 3
    #strideSize = 1
    seqSize = len(seq)
    start = 0

    listSubSeqs = []
    while start + (windowSize - 1) <= (seqSize -1):
        listSubSeqs.append(seq[start : start + windowSize])
        start = start + strideSize
    
    if printInfo:
        print(listSubSeqs)
        print(seqSize)
        print(len(listSubSeqs))

    return listSubSeqs

def organizeSubSeqs(listSubSeqs, regionSize = 2):

    genMatrix = np.empty((0,regionSize))
    #regionSize = regionSize
    subseqIndex = 0
    subSeqLength = len(listSubSeqs)
    subSeqLastIndex = subSeqLength - 1
    while subseqIndex <= (subSeqLastIndex - (regionSize - 1)):
        newColumnStart = subseqIndex
        newColumn = np.array([])
        while newColumnStart < subseqIndex + regionSize:
            newColumn = np.append(newColumn, listSubSeqs[newColumnStart])   #listSubSeqs[newColumnStart])
            newColumnStart = newColumnStart + 1
        genMatrix = np.vstack((genMatrix, newColumn))
        subseqIndex = subseqIndex + 1

    return genMatrix


def genBitMatrixFromSubSeqMatrix(subSeqMatrix, dictSubSeqToArrayBit, colSize = 16):
    num_lines = subSeqMatrix.shape[0]
    num_cols = subSeqMatrix.shape[1]
    bitsMatrix = np.empty((0,colSize))  #TODO: fixo


    for l in range(0, num_lines):
        bitsLine = np.array([])
        for c in range(0, num_cols):
            #print(subSeqMatrix[l][c])
            bitsLine = np.append(bitsLine, dictSubSeqToArrayBit[subSeqMatrix[l][c]])
        
        bitsMatrix = np.vstack((bitsMatrix, bitsLine))

    return bitsMatrix


def verifyIfSeqWindowAndStrideMatches(seqSize, windowSize, strideSize):
    last_window_start = 0
    nOfStrides = 0
    while (last_window_start + windowSize - 1) < (seqSize - 1):
        last_window_start = strideSize * nOfStrides
        nOfStrides = nOfStrides + 1
        #print(last_window_start)
        #print(nOfStrides)
        #print(' ')

    print('n of windows: ' + str(nOfStrides))
    print('last window start: ' + str(last_window_start))
    print('last window end: ' + str(last_window_start + windowSize - 1))
    print('max seq position: ' + str(seqSize - 1))


def salvar_pickle(objeto, nome_arquivo):
  
  with open(nome_arquivo, 'wb') as arquivo:
    pickle.dump(objeto, arquivo)

def carregar_pickle(nome_arquivo):
  with open(nome_arquivo, 'rb') as arquivo:
    objeto = pickle.load(arquivo)
  return objeto




In [8]:
h3df = pd.read_csv('h3k4me2_csv.csv');
h3df = h3df.drop(columns= 'Unnamed: 0')
h3df = h3df.drop(columns= 'seqSize')
h3df
# promoters.iloc[0,2]

# linesSize = promoters.shape[0]

# for line in range(linesSize):
#     promoters.iloc[line, 2] = promoters.iloc[line, 2].replace('\t', '')
#     if(promoters.iloc[line, 0] == '+'):
#         promoters.iloc[line, 0] = '1'
#     else:
#         promoters.iloc[line, 0] = '0'

# promoters

,seqName,sequence,classe
0,>YEL016C_YEL016C_125921,CAGTTTGCAGTAGTTGCCAAATAGGGTCAGCCGTGTTGCTCCAAAT...,1
1,>iYBR044C_326466,TACCTAAACTATTAAAAGCGGACCCATCTAAAACATAACGTTCGGC...,1
2,>YLL057C_JLP1_26684,TGAGGGTGGCCACTAGTTTGATGAATGTGCAACTTGCCAAAATGTC...,1
3,>YIL070C_MAM33_230663,TCGTCCAAGTTTGAAAAAGGTGGGCCATGGTATACCAATTCTCTTG...,1
4,>YJR070C_LIA1_569636,AGATTCTCTGACGACATCGACTTCATCGTTGAGGTAAGATTTCAAG...,1
...,...,...,...
25073,>YPR198W_SGE1_935256,ATTTAGTCCAGCTTTATCAACTAGTATTTAAAAAAAAGCCTACATT...,0
25074,>YPR198W_SGE1_935551,AACATCAGACGATCCAGACTTTCAAAACAAGTTTATTGAAGTCACA...,0
25075,>iYPR198W_935756,TATGATGGCCCCCAATCTTCACTATCAAAGTTCATAAACACAGCTA...,0
25076,>iYPR198W_936110,TCATAATTACAAAATACCCTGTGATTAGGATATCATTATTACTAAC...,0


In [10]:
seqList = h3df['sequence'].to_numpy()
classList = h3df['classe'].to_numpy()

#### calculando tamanhos

In [17]:
verifyIfSeqWindowAndStrideMatches(500, 3, 1)   #500, 3, 1; 500,15, 5

n of windows: 498
last window start: 497
last window end: 499
max seq position: 499


In [21]:
dictH3k4me2 = createDictForNucleotidesAlphabet(['A','C','G','T'], 12, 25)

In [12]:
dictH3k4me2

{'AAA': [0, 0, 0, 0, 0, 0, 0, 1],
 'AAC': [0, 0, 0, 0, 0, 0, 1, 0],
 'AAG': [0, 0, 0, 0, 0, 0, 1, 1],
 'AAT': [0, 0, 0, 0, 0, 1, 0, 0],
 'ACA': [0, 0, 0, 0, 0, 1, 0, 1],
 'ACC': [0, 0, 0, 0, 0, 1, 1, 0],
 'ACG': [0, 0, 0, 0, 0, 1, 1, 1],
 'ACT': [0, 0, 0, 0, 1, 0, 0, 0],
 'AGA': [0, 0, 0, 0, 1, 0, 0, 1],
 'AGC': [0, 0, 0, 0, 1, 0, 1, 0],
 'AGG': [0, 0, 0, 0, 1, 0, 1, 1],
 'AGT': [0, 0, 0, 0, 1, 1, 0, 0],
 'ATA': [0, 0, 0, 0, 1, 1, 0, 1],
 'ATC': [0, 0, 0, 0, 1, 1, 1, 0],
 'ATG': [0, 0, 0, 0, 1, 1, 1, 1],
 'ATT': [0, 0, 0, 1, 0, 0, 0, 0],
 'CAA': [0, 0, 0, 1, 0, 0, 0, 1],
 'CAC': [0, 0, 0, 1, 0, 0, 1, 0],
 'CAG': [0, 0, 0, 1, 0, 0, 1, 1],
 'CAT': [0, 0, 0, 1, 0, 1, 0, 0],
 'CCA': [0, 0, 0, 1, 0, 1, 0, 1],
 'CCC': [0, 0, 0, 1, 0, 1, 1, 0],
 'CCG': [0, 0, 0, 1, 0, 1, 1, 1],
 'CCT': [0, 0, 0, 1, 1, 0, 0, 0],
 'CGA': [0, 0, 0, 1, 1, 0, 0, 1],
 'CGC': [0, 0, 0, 1, 1, 0, 1, 0],
 'CGG': [0, 0, 0, 1, 1, 0, 1, 1],
 'CGT': [0, 0, 0, 1, 1, 1, 0, 0],
 'CTA': [0, 0, 0, 1, 1, 1, 0, 1],
 'CTC': [0, 0,

In [22]:
dictMatrixBySeq = {}
progressCounter = 0;
for seq in seqList:
    subSeqsList = generateSubSeqsFromSeq(seq, 12, 4) #def generateSubSeqsFromSeq(seq, windowSize = 3, strideSize = 1, printInfo = False)
    subSeqMatrix = organizeSubSeqs(subSeqsList)
    bitsMatrix = genBitMatrixFromSubSeqMatrix(subSeqMatrix, dictH3k4me2, colSize = 50)
    dictMatrixBySeq[seq] = bitsMatrix
    progressCounter = progressCounter + 1;
    print(str(progressCounter) + '/' + str(len(seqList)) + ' sequências processadas')

dictMatrixBySeq

1/25078 sequências processadas
2/25078 sequências processadas
3/25078 sequências processadas
4/25078 sequências processadas
5/25078 sequências processadas
6/25078 sequências processadas
7/25078 sequências processadas
8/25078 sequências processadas
9/25078 sequências processadas
10/25078 sequências processadas
11/25078 sequências processadas
12/25078 sequências processadas
13/25078 sequências processadas
14/25078 sequências processadas
15/25078 sequências processadas
16/25078 sequências processadas
17/25078 sequências processadas
18/25078 sequências processadas
19/25078 sequências processadas
20/25078 sequências processadas
21/25078 sequências processadas
22/25078 sequências processadas
23/25078 sequências processadas
24/25078 sequências processadas
25/25078 sequências processadas
26/25078 sequências processadas
27/25078 sequências processadas
28/25078 sequências processadas
29/25078 sequências processadas
30/25078 sequências processadas
31/25078 sequências processadas
32/25078 sequênci

{'CAGTTTGCAGTAGTTGCCAAATAGGGTCAGCCGTGTTGCTCCAAATTCTTGCGTCGAGGTTATTTGGTCTAAATTCGCTACTCGTGAAATTATCCCAGAATATATTGGAAACAATACCGTGCTCAATGGGATATTTTCCCGTTACCATGCTCCAGTGGTTGGGAAATGTCTGAGTGGGAAAACTTGGGATCATATACGGTGCAGTCGTGATATTCATATCGTACGGGGACCGCAAATTGTGCAAGTTGTAAAGAAATGGCGTGTATTTTGCATCTATCAATCTCGGATGGAACCCATCGATAGATATCAATATCGTTAACGTCTTGAAATCATGTGTGCCGTTGAAGTAAGTATTTGATCTATTCGCAATTGTCCGGGGCCCATTATCAATAGGTTTTGGGCACTTCGAATATAAAAGCAACTCAATGAGTATTATACCGCAAAGCAAAAGTTTCAATAAAAAATGTCGTGATATTGCGAAAGGTTTTATGTTCGTATCA': array([[0., 0., 1., ..., 1., 1., 1.],
        [0., 1., 1., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 1., 1.],
        ...,
        [0., 1., 0., ..., 1., 0., 0.],
        [0., 0., 0., ..., 1., 0., 0.],
        [0., 1., 1., ..., 1., 0., 1.]]),
 'TACCTAAACTATTAAAAGCGGACCCATCTAAAACATAACGTTCGGCAAGTAGATGTATTGTTCCAAAAAGGCAATACATCCATACAACGGTGCCACCGATCTGTAATGAATGCTCCGTAATGTTTGGGGGATTCATTAAATTTACTGCTGGAACCGAGAGAGAAACACGACAGAGCAATTCATATAAATGAATAAATGAAAACTCGGTCGTATTGCCCTTAAACGTTTATAGGACCAGGTTGGCT

In [23]:
keyList = list(dictMatrixBySeq.keys())
#imagesHumanDataSetMatrix
counter = 0
while counter < 15:
    key = keyList[counter]
    bitsMatrix = dictMatrixBySeq[key]
    img = Image.fromarray(np.uint8(bitsMatrix * 255) , 'L')
    img.save('imagesH3k4me2w12s4/'+key[0:15]+'.png')
    counter = counter + 1

In [43]:
keyList = list(dictMatrixBySeq.keys())

index = 0

while index < 10:
    key = keyList[index]
    bitsMatrix = dictMatrixBySeq[key]
    img = Image.fromarray(np.uint8(bitsMatrix * 255) , 'L')
    img.save('imagesH3S4W12/'+key[0:15]+'.png')
    index = index + 1

#### montando dataset

In [24]:
listClasses = []
listMatrix = []
listSequence = []
dictToExport = {}

sequences = list(dictMatrixBySeq.keys())

for sequence in sequences:
    
    qr = 'sequence == ' + '"' + sequence + '"'
    strClass = h3df.query(qr)['classe'].iloc[0]
    valClass = int(strClass)

    matrix = dictMatrixBySeq[sequence]

    listClasses.append(valClass)
    listMatrix.append(matrix)
    listSequence.append(sequence)

listMatrixNp = np.array(listMatrix)
listClassesNp = np.array(listClasses)

dictToExport['inputs'] = listMatrixNp
dictToExport['outputs'] = listClassesNp
dictToExport['sequences'] = listSequence
    

In [25]:
salvar_pickle(dictToExport, 'dataset_H3k4me2_W12S4.pkl')